In [1]:
using QuantumOptics
using CollectiveSpins
using Random

In [2]:
""" Return H_ZZ for the spin collection S """
function compute_H_ZZ(Nx, Ny, Nz, bMB, Ω_ZZ_coeff)
    H = SparseOperator(bMB)
    Matidx = reshape([1:N;], Nx, Ny, Nz)
    
    for i=1:N
        idx = findall(x->x==i, Matidx)[1]
        x, y, z = idx[1], idx[2], idx[3]

        # Neighbourg displacment
        neighbors = [
        (1, 0, 0), (-1, 0, 0),
        (0, 1, 0), (0, -1, 0),
        (0, 0, 1), (0, 0, -1)
        ]

        for (dx, dy, dz) in neighbors
            xj, yj, zj = x + dx, y + dy, z + dz
            # Check if neighbours are on the lattice
            if 1 <= xj <= Nx && 1 <= yj <= Ny && 1 <= zj <= Nz
                j = LinearIndices((Nx, Ny, Nz))[xj, yj, zj]
                
                H += (Ω_ZZ_coeff[1] * embed(bMB, i, σp*σm)*embed(bMB, j, σp*σm) + 
                    Ω_ZZ_coeff[2] * (embed(bMB, i, σp*σm)*embed(bMB, j, σm*σp) + embed(bMB, i, σm*σp)*embed(bMB, j, σp*σm)) + 
                    Ω_ZZ_coeff[3] * embed(bMB, i, σm*σp)*embed(bMB, j, σm*σp))
            end
        end
    end
    return H
end

""" Compute the spin flip exchange hamiltonian """
function compute_H_SFE(N, bMB, Ω_CSC)
    H = SparseOperator(bMB)
    
    for i=1:N, j=1:N
        if i==j
            continue
        end
        
        H += Ω_CSC[i, j] * embed(bMB, i, σp)*embed(bMB, j, σm)
        
    end
    return H
end

""" Prepare the initial state of the system 
with a phase correlation imposed between the atoms
because the laser arrive on the lattice with an angle θl """
function prepare_phi_IS(θl, λl, theta_array)
    phi_array = zeros(N)
    for i = 1:N
        phi_array[i] = 2pi * sqrt(sum((system.spins[i].position .* [cos(θl), sin(θl), 0]) .^ 2))
    end
    return phi_array
end

prepare_phi_IS

In [3]:
# Define geometry of system
Nx,Ny,Nz = [2,1,1]
N = Nx*Ny*Nz
NES = N

# Normalization variables
λl = 1299. # In nm
Γ0 = 1. # In Hz

# Creating the lattice
a_dim, b_dim, c_dim = [266., 266., 532.]./λl # Dimension of the lattice normalized
geo = CollectiveSpins.geometry.box(a_dim,b_dim,c_dim;Nx=Nx,Ny=Ny,Nz=Nz)
e = [0,0,1]   # Quantization axis
system = CollectiveSpins.SpinCollection(geo, e, gammas=1.)

# Initial state
θl = 30*pi/180
theta = π/2
theta_array = shuffle(vcat(ones(Int, NES)*theta, ones(Int, N-NES)*π))
phi_array = prepare_phi_IS(θl, λl, theta_array)

# Constante variable
Ω_ZZ_coeff = [53, 42, 33]./Γ0 # Normalized by Γ0, [up/up (-7/-7), up/down (-7/-6), down, down (-6/-6)]

# Integration time
Tstep, Tend = 1e-3, 1 # Normalised time (t/τ)
T = [0:Tstep:Tend;];

In [4]:
# Basis
b1 = SpinBasis(1//2)
bMB = CompositeBasis([b1 for i = 1:N])

# Single atom operators
σx = sigmax(b1)
σy = sigmay(b1)
σz = sigmaz(b1)
σp = sigmap(b1)
σm = sigmam(b1)

# Single atoms σz for a -6/-7 system
σz67 = -7*(σp*σm) -6*(σm*σp)

# Many body operators
Jx = sum([embed(bMB, i, σx) for i = 1:N])
Jy = sum([embed(bMB, i, σy) for i = 1:N])
Jz67 = sum([embed(bMB, i, σz67) for i = 1:N]);

In [5]:
Ω_CS = CollectiveSpins.OmegaMatrix(system)
Γ_CS = CollectiveSpins.GammaMatrix(system)

# Hamiltonian
HCS = compute_H_SFE(N, bMB, Ω_CS)
HZZ = compute_H_ZZ(Nx, Ny, Nz, bMB, Ω_ZZ_coeff)
H = HCS + HZZ

# Construct Jump operators
J = [embed(bMB, i, σm) for i = 1:N];

In [6]:
Ψ0 = CollectiveSpins.quantum.blochstate(phi_array, theta_array)
@time begin
    ρt = timeevolution.master(T, Ψ0, H, J, rates=Γ_CS, dt=Tstep)[2];
end
nothing

  4.767577 seconds (19.46 M allocations: 1004.980 MiB, 10.48% gc time, 99.97% compilation time)
